# Running a 5-Point Calibration Scan With OSO

This notebook adapts the Five Point Calibration Scan notebook to work with PSI. Running this notebook requires that a PSI namespace be spun up with `DISH_LMC_ENABLED` set to true.

This notebook can be Run with BITE or SPFRx, with BITE requiring the optional BITE notebook to be run. 

## 1 Setup 

### 1.1 Environment Setup

In [1]:
import sys

sys.path.append("../../src")

import json
import os
from time import sleep

from ska_oso_pdm import SBDefinition
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions import pdm_transforms
from ska_oso_scripting.functions.sb import create_sbi, load_sbd
from ska_oso_scripting.objects import SubArray, Telescope
from tango import DevFailed, DeviceProxy

import notebook_tools.wait_for_tango as wait_for_tango

### 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [3]:
!kubectl get ns | grep ska-mid-psi 

ci-ska-mid-psi-1386660462-alexschell                  Active   8m6s
ci-ska-mid-psi-1386660462-alexschell-sdp              Active   8m7s
ci-ska-mid-psi-1390816548-amjoshi                     Active   38h
ci-ska-mid-psi-1390816548-amjoshi-sdp                 Active   38h


And load it into the variables for the notebook:

In [4]:
namespace = "ci-ska-mid-psi-1386660462-alexschell"  # set to desired NS
simulation_mode = 0  # set to 1 to run in sim mode
target_boards_list = [1]  # assign boards
test_id = "talon2 basic gaussian noise"  # Test to send config from
server = "ska-sdp-kafka." + namespace + ".svc.cluster.local:9092"
subarray_id = 1

Get the SKUID pod:

In [5]:
!kubectl get pods -n $namespace | grep skuid

ska-ser-skuid-test-1386660462-58bb66f6f8-mvfpm                   1/1     Running     0          7m12s


And set the id value based on the name of this pod:

In [6]:
skuid_id = "ska-ser-skuid-test-1386660462-svc"
kafka_id = ""

Next, load all the other vars the notebook will use. These should not need to be changed for this run.

In [31]:
# Fully Qualified Domain Names for the devices to set up proxies
CSP_CONTROLLER_FQDN = "mid-csp/control/0"
CSP_SUBARRAY_FQDN = "mid-csp/subarray/01"


CENTRAL_NODE_FQDN = "ska_mid/tm_central/central_node"
LEAF_NODE_SUBARRAY_FQDN = "ska_mid/tm_leaf_node/csp_subarray01"
TMC_SUBARRAY_FQDN = "ska_mid/tm_subarray_node/1"

LEAF_NODE_MASTER_FQDN = "ska_mid/tm_leaf_node/csp_master"

CBF_SUBARRAY_FQDN = "mid_csp_cbf/sub_elt/subarray_01"

BITE_FQDN = "mid_csp_cbf/ec/bite"
DEPLOYER_FQDN = "mid_csp_cbf/ec/deployer"

# Tango host environment variable
TANGO_HOST = "databaseds-tango-base." + namespace + ".svc.cluster.local:10000"
SKUID_POD = skuid_id + "." + namespace + ".svc.cluster.local:5004"

# For now, use direct cluster IP/port access for kafka and sku id pods
SKUID_POD = "10.10.47.134:9870"
KAFKA_POD = "10.110.145.7:9092"

# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf")
CSP_CONFIG = os.path.join(DATA_DIR, "mid_telescope/csp")
TMC_CONFIG = os.path.join(DATA_DIR, "mid_telescope/tmc")
HW_CONFIG = os.path.join(DATA_DIR, "psi/cbf/hw_config")
SLIM_CONFIG = os.path.join(COMMON_CONFIG, "slim_config")
CBF_INPUT_DIR = os.path.join(COMMON_CONFIG, "cbf_input_data")
# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]

Next, set the environment arg for TANGO HOST, SKUID_POD and KAFKA, along with the oda API service endpoint:

In [11]:
os.environ["TANGO_HOST"] = TANGO_HOST
os.environ["SKUID_URL"] = SKUID_POD
os.environ["ODA_URL"] = f"http://142.73.34.170/{namespace}/oda/api/v5/"
os.environ["telescope"] = "mid"
print("SKUID service set to: ", os.environ["SKUID_URL"])
print("TANGO_HOST set to:", os.environ["TANGO_HOST"])
print("ODA endpoint set to:{}".format(os.environ["ODA_URL"]))
print("This can be checked via {}ui".format(os.environ["ODA_URL"]))

SKUID service set to:  10.101.212.237:9870
TANGO_HOST set to: databaseds-tango-base.ci-ska-mid-psi-1386660462-alexschell.svc.cluster.local:10000
ODA endpoint set to:http://142.73.34.170/ci-ska-mid-psi-1386660462-alexschell/oda/api/v5/
This can be checked via http://142.73.34.170/ci-ska-mid-psi-1386660462-alexschell/oda/api/v5/ui


With all the file paths defined the JSON files can be loaded in and checked.

In [12]:
print("Getting files...")

INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG, "sys_params/initial_system_param.json")
ASSIGN_RESOURCES_FILE = os.path.join(TMC_CONFIG, "assign_resources.json")
CONFIGURE_SCAN_FILE = os.path.join(TMC_CONFIG, "configure_scan.json")
SCAN_FILE = os.path.join(TMC_CONFIG, "scan.json")

CBF_INPUT_FILE = f"{CBF_INPUT_DIR}/cbf_input_data.json"
BITE_CONFIG_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/bite_configs.json"
FILTERS_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/filters.json"

DISH_CONFIG_FILE = f"{COMMON_CONFIG}/sys_params/load_dish_config.json"

START_CHANNEL = 0
END_CHANNEL = 14860
START_PORT = 21000

SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
    CBF_INPUT_FILE,
    BITE_CONFIG_FILE,
    FILTERS_FILE,
    DISH_CONFIG_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

Getting files...
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/sys_params/initial_system_param.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/configure_scan.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/scan.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/cbf_input_data/cbf_input_data.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/cbf_input_data/bite_config_parameters/bite_configs.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/cbf_input_data/bite_config_parameters/filters.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/sys_params/load_dish_config.json exists: ✔️


Next, set up the hw config to match the boards in use:

In [13]:
if any(i > 4 for i in target_boards_list):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_boards_list = list(map(lambda x: x - 4, target_boards_list))

else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

receptor_ids = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list))

Using standard HW config
HW config: ✔️


If the HW config file exists, it is loaded into the pod:

In [14]:
!kubectl cp $HW_CONFIG_FILE $namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml 

### 1.3 Create Device Proxies

Using the FQDNs set earlier, and with the pod spun up, create device proxies to the devices used and check the connection to them.

In [15]:
# CSP Devices
csp_controller = DeviceProxy(CSP_CONTROLLER_FQDN)
print(f"CSP Controller: {csp_controller.Status()}")
csp_subarray = DeviceProxy(CSP_SUBARRAY_FQDN)
print(f"CSP Subarray: {csp_subarray.Status()}")

# TMC Devices
tmc_central_node = DeviceProxy(CENTRAL_NODE_FQDN)
print(f"Central Node: {tmc_central_node.Status()}")
leaf_node_subarray = DeviceProxy(LEAF_NODE_SUBARRAY_FQDN)
print(f"Leaf Subarray Node: {leaf_node_subarray.Status()}")
tmc_subarray = DeviceProxy(TMC_SUBARRAY_FQDN)
print(f"TMC subarray Node: {tmc_subarray.Status()}")

leaf_node_master = DeviceProxy(LEAF_NODE_MASTER_FQDN)

# Deployer for setup and BITE for data mocking
bite = DeviceProxy(BITE_FQDN)
deployer = DeviceProxy(DEPLOYER_FQDN)

sdp_subarray = DeviceProxy("mid-sdp/subarray/01")

# CBF Device
cbf_subarray = DeviceProxy(CBF_SUBARRAY_FQDN)
print(f"CBF subarray: {cbf_subarray.Status()}")

CSP Controller: The device is in DISABLE state.
CSP Subarray: The device is in DISABLE state.
Central Node: The device is in ON state.
Leaf Subarray Node: The device is in ON state.
TMC subarray Node: The device is in ON state.
CBF subarray: The device is in DISABLE state.


### 1.4 Downloading Requirements via the Deployer

First, set the board to deploy to and turn on the deployer device.

In [16]:
deployer.targetTalons = target_boards_list
print(deployer.targetTalons)
deployer.generate_config_jsons()

[1]


Once started and configured, the required devices can then be downloaded.

In [17]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

Now configure the device database with the downloaded devices:

In [18]:
deployer.configure_db()

### 1.5 Set up Execution Block

The first step is to generate an empty Execution Block, using the help to work with the API endpoint:

In [19]:
exec_block_id = oda_helper.create_eb(telescope="ska_mid")
print(exec_block_id)

eb-t0001-20240729-00002


## 2 Preparing Telescope

Start by setting the admin modes to allow for control of the telescope:

In [20]:
csp_controller.write_attribute("adminMode", 0)
tmc_central_node.write_attribute("adminMode", 0)
# Set CBF Simulation mode to false and CBF timeout to 99s
csp_controller.cbfSimulationMode = 0
csp_controller.commandTimeout = 99

Start by loading in the VCC config (this step may need to be re-run):

In [24]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))

sleep(2)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{leaf_node_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{leaf_node_master.sourceDishVccConfig}"
)

dish_config_json file contents: 
{'interface': 'https://schema.skao.int/ska-mid-cbf-initsysparam/1.0', 'tm_data_sources': ['car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata'], 'tm_data_filepath': 'instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json'}
TMC CSP Master's Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 1}, "SKA036": {"vcc": 2, "k": 1}, "SKA063": {"vcc": 3, "k": 1}, "SKA100": {"vcc": 4, "k": 1}}}

TMC CSP Master's Source Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"], "tm_data_filepath": "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"}


Before continuing, ensure this passes to make sure the DishVCC is set to allow for the On command to be sent.

In [25]:
if tmc_central_node.isDishVccConfigSet:
    print("DishVCC set, can continue!")
else:
    print("DishVCC not set, repeat last step.")

DishVCC set, can continue!


Set up the objects for controlling the telescope via OSO:

In [26]:
sub = SubArray(subarray_id)
tel = Telescope()

Now, while monitoring the Taranta dashboard, send the ON command to the telescope and monitor that the LURs Controller and subarrays come on:

In [27]:
tel.on()

1|2024-07-29T18:55:56.097Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track telescopeState of ska_mid/tm_central/central_node
1|2024-07-29T18:55:56.124Z|INFO|MainThread|execute|tango_executor.py#243||Executing command: <Command('ska_mid/tm_central/central_node', 'TelescopeOn', )>
1|2024-07-29T18:55:56.134Z|INFO|MainThread|wait_for_transition|common.py#338||Waiting for telescopeState to transition to ON
1|2024-07-29T18:55:56.135Z|INFO|MainThread|read_event|tango_executor.py#184||timeout None to wait for read an event from queue
1|2024-07-29T18:55:56.159Z|INFO|MainThread|read_event|tango_executor.py#184||timeout None to wait for read an event from queue
1|2024-07-29T18:55:57.140Z|INFO|MainThread|wait_for_transition|common.py#354||telescopeState reached target state ON


### 2.1 Set up CSP and TMC

In [28]:
print(f"CSP controller state is: {csp_controller.state()}")
print(f"Central node state is: {tmc_central_node.state()}")

CSP controller state is: ON
Central node state is: ON


### 2.2 Define Scheduling Block

Temporary work around for ingress issues:

In [32]:
os.environ["SKUID_URL"] = SKUID_POD
print("SKUID service set to: ", os.environ["SKUID_URL"])

SKUID service set to:  10.10.47.134:9870


Now, use the SKUID service to generate processing and execution block IDs from the scheduling block definition.

In [33]:
sb = load_sbd("./sb/mid_5_point_sb.json")
sbi = create_sbi(sb)

sbi.sdp_configuration.execution_block.eb_id = exec_block_id

1|2024-07-29T18:58:15.588Z|INFO|MainThread|create_sbi|sb.py#28||New SBI ID mapping: sbi-mvp01-20200325-00001 -> sbi-t0001-20240729-00003
1|2024-07-29T18:58:15.594Z|INFO|MainThread|create_sbi|sb.py#41||New PB ID mapping: pb-test-20211111-00000 -> pb-t0001-20240729-00004
1|2024-07-29T18:58:15.594Z|INFO|MainThread|create_sbi|sb.py#41||New PB ID mapping: pb-test-20211111-00001 -> pb-t0001-20240729-00005


In [34]:
sb_json = SBDefinition.model_dump_json(sb)
sb_json = sb_json.replace("<KAFKA_HOST>", KAFKA_POD)
sb = SBDefinition.model_validate_json(sb_json)

## 3 Running the Scan

### 3.1 Assigning Resources

Now, the resources can be assigned to the telescope, either via getting the resources from the scheduling block:

In [35]:
assign_requests = pdm_transforms.create_cdm_assign_resources_request_from_scheduling_block(
    sub.id, sb
)

1|2024-07-29T18:58:35.403Z|INFO|MainThread|create_mid_assign_resources_request|wrapper.py#81||Setting dish : frozenset({'SKA100', 'SKA001', 'SKA063', 'SKA036'}) 
1|2024-07-29T18:58:35.404Z|INFO|MainThread|convert_channels|sdp.py#293||Setting ChannelConfiguration id:vis_channels , spectral_windows:[SpectralWindow(spectral_window_id='fsp_1_channels', count=32, start=0, freq_min=1326000000.0, freq_max=1367000000.0, stride=1, link_map=[(0, 0), (200, 1), (744, 2), (944, 3)])] 
1|2024-07-29T18:58:35.404Z|INFO|MainThread|convert_spectral_window|sdp.py#272||Setting channel attribute -> count:32 , start:0 , stride:1, freq_min:1326000000.0, freq_max:1367000000.0 , link_map:[(0, 0), (200, 1), (744, 2), (944, 3)] 
1|2024-07-29T18:58:35.405Z|INFO|MainThread|convert_target_to_fieldconfiguration|sdp.py#347||Setting target attribute -> target_id:Polaris Australis , reference_coordinate:<EquatorialCoordinates: (21h08m47.92s -88d57m22.9s ICRS)> 
1|2024-07-29T18:58:35.407Z|INFO|MainThread|convert_target_

In [42]:
print(assign_requests)
print(assign_requests[0])
stripped_assign_requests = assign_requests[0]
stripped_assign_requests.interface = ""
stripped_assign_requests.dish.receptor_ids = {"SKA001"}
print(stripped_assign_requests)

[AssignResourcesRequest(subarray_id=1, dish=DishAllocation(receptor_ids=frozenset({'SKA001'})), sdp_config=SDPConfiguration(eb_id=None, max_length=None, scan_types=None, processing_blocks=[ProcessingBlockConfiguration(pb_id='pb-test-20211111-00000', workflow=None, parameters={'version': '0.4.0', 'num_scan': 5, 'additional_args': ['--thresh_width', '1.25'], 'kafka_topic': 'pointing_offset'}, dependencies=[PbDependency(pb_id='pb-test-20211111-00001', kind=['vis-receive'])], sbi_ids=[], script=ScriptConfiguration(kind='realtime', name='pointing-offset', version='0.4.0')), ProcessingBlockConfiguration(pb_id='pb-test-20211111-00001', workflow=None, parameters={'channels_per_port': 32, 'queue_connector_configuration': {'exchanges': []}, 'transport_protocol': 'tcp', 'extra_helm_values': {'receiver': {'options': {'reception': {'reset_time_indexing_after_each_scan': True, 'stats_receiver_kafka_config': '10.110.145.7:9092:json_workflow_state'}, 'telescope_model': {'telmodel_key': 'instrument/ska

TODO: Get around channel count validation error for this step, timeout can occur but everything can be set.

In [43]:
sub.assign_from_cdm(stripped_assign_requests, timeout=200)

1|2024-07-29T20:20:46.465Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track obsState of ska_mid/tm_subarray_node/1
1|2024-07-29T20:20:46.469Z|INFO|MainThread|_get_id_from_params_or_generate_new_id|transactions.py#172||Generated transaction ID txn-t0001-20240729-000000010
1|2024-07-29T20:20:46.470Z|INFO|MainThread|log_entry|transactions.py#132||Transaction[txn-t0001-20240729-000000010]: Enter[AssignResources] with parameters [{}] marker[82420]
1|2024-07-29T20:20:46.470Z|INFO|MainThread|_call_and_wait_for_transition|common.py#178||Added Transaction_id to command: <Command('ska_mid/tm_central/central_node', 'AssignResources', '{"subarray_id": 1, "dish": {"receptor_ids": ["SKA001"]}, "sdp": {"processing_blocks": [{"pb_id": "pb-test-20211111-00000", "parameters": {"version": "0.4.0", "num_scan": 5, "additional_args": ["--thresh_width", "1.25"], "kafka_topic": "pointing_offset"}, "dependencies": [{"pb_id": "pb-test-20211111-00001", "kind": ["vis-receive"]}]

EventTimeoutError: Read event timeout from queue occured while execution of AssignResources command

Or via manual setting by JSON.

In [ ]:
print(sdp_subarray.state())

sleep(3)

print(
    "Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = ["SKA001"]
    assign_resources_json["sdp"]["resources"]["receptors"] = ["SKA001"]


print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

tmc_subarray.AssignResources(json.dumps(assign_resources_json))

sleep(2)
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")
print(f"Assign Resource: {assign_resources_json}")

### 3.2 Configure the Scans and Run

In [44]:
print(sb.scan_sequence)

['calibrator scan', 'science scan', 'science scan', 'calibrator scan']
